## 1.Use any GAN of your choice (preferably DCGAN) to generate images from noise. Perform the following experiments.

### A. Use the CIFAR 10 database to learn the GAN network. Generate images once the learning is complete.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Define the generator and discriminator networks
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)

# Set hyperparameters
nz = 100  # Size of the input noise vector
ngf = 64  # Number of generator filters
ndf = 64  # Number of discriminator filters
nc = 3    # Number of channels (RGB)
lr = 0.0002
batch_size = 128
num_epochs = 100

# Create the generator and discriminator
generator = Generator(nz, ngf, nc)
discriminator = Discriminator(nc, ndf)

# Initialize weights
generator.apply(weights_init)
discriminator.apply(weights_init)

# Define loss and optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Load the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = datasets.CIFAR10(root='./data', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        real_images, _ = data
        batch_size = real_images.size(0)
        
        # Update the discriminator
        discriminator.zero_grad()
        real_labels = torch.full((batch_size,), 1.0)
        fake_labels = torch.full((batch_size,), 0.0)
        noise = torch.randn(batch_size, nz, 1, 1)
        
        # Generate fake images
        fake_images = generator(noise)
        
        # Discriminator's loss for real and fake images
        output_real = discriminator(real_images)
        output_fake = discriminator(fake_images.detach())
        
        loss_real = criterion(output_real, real_labels)
        loss_fake = criterion(output_fake, fake_labels)
        loss_d = loss_real + loss_fake
        
        loss_d.backward()
        optimizer_d.step()
        
        # Update the generator
        generator.zero_grad()
        output_fake = discriminator(fake_images)
        loss_g = criterion(output_fake, real_labels)
        loss_g.backward()
        optimizer_g.step()
        
        if i % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Batch [{i}/{len(dataloader)}], Loss D: {loss_d.item()}, Loss G: {loss_g.
                  item()}')

# Generate and save some fake images
noise = torch.randn(64, nz, 1, 1)
fake_images = generator(noise)
fake_images = fake_images.detach().numpy()

# Display generated images
plt.figure(figsize=(10, 10))
for i in range(64):
    plt.subplot(8, 8, i + 1)
    plt.axis('off')
    plt.imshow(np.transpose(fake_images[i], (1, 2, 0))

plt.show()

### B. Plot generator and discriminator losses and show how can you ascertain the convergence of the GAN training process.

In [ ]:
Training a Generative Adversarial Network (GAN) involves monitoring the losses of both the generator and discriminator to 
assess the convergence of the training process. I'll provide a general outline of how to do this using DCGAN (Deep 
Convolutional GAN) as an example, but the specific implementation may vary depending on the framework you are using (e.g.,
TensorFlow, PyTorch).

Here's a step-by-step guide to plot the generator and discriminator losses and ascertain the convergence of the GAN training
process:

1.Initialize the GAN: Set up the generator and discriminator networks, along with their respective optimizers. Ensure that
you have a well-defined architecture for both networks. In the case of DCGAN, convolutional layers are commonly used for 
better performance.

2.Define Loss Functions:

    ~Generator Loss: Typically, the generator minimizes the binary cross-entropy loss. The loss quantifies how well the 
    generator fools the discriminator.
    ~Discriminator Loss: The discriminator minimizes the binary cross-entropy loss to distinguish between real and fake 
    images.
    
3.Training Loop:

    ~In each iteration, generate a batch of random noise (randomly sampled from a normal distribution) and use the generator
    to create fake images.
    ~Combine these fake images with real images from your dataset.
    ~Calculate the discriminator loss on the combined dataset.
    ~Update the discriminator's weights using backpropagation and the discriminator optimizer.
    ~Calculate the generator loss, which is based on the output of the discriminator when the fake images are passed through.
    ~Update the generator's weights using backpropagation and the generator optimizer.
    
4.Log Losses:

    ~During training, log the discriminator loss and generator loss at regular intervals (e.g., after each epoch or after
     a certain number of batches).
    
5.Plot Losses:

    ~Use a plotting library (e.g., Matplotlib) to create two separate plots for the discriminator loss and generator loss 
    over time.
    
6.Convergence Criteria:

    ~Convergence of a GAN can be assessed by observing the loss curves. In general, you want to see the following:
        ~The discriminator loss should decrease and stabilize as the discriminator becomes better at distinguishing real
        from fake.
        ~The generator loss should decrease and stabilize as the generator becomes better at producing realistic images.
        ~The two loss curves should reach a balance, with neither the generator nor discriminator dominating. This 
        equilibrium indicates convergence.
        
7.Early Stopping (optional):

    ~You can implement an early stopping mechanism based on your loss curves. If the losses do not show signs of convergence
    (e.g., they keep fluctuating or diverging), you might want to stop training.
    
8.Visual Inspection:

    ~Additionally, you can visually inspect the generated images over time to see if they improve in quality and realism.
    
The key to ensuring convergence is to carefully monitor the loss curves and experiment with different hyperparameters and 
network architectures until you achieve stable and realistic image generation.

Remember that GAN training can be sensitive to hyperparameters like learning rates, architecture choices, and batch sizes.
Experimenting with these parameters is often necessary to achieve good results.

## 2.Fine-tuning Take a ResNet50 model and the database to be used for this question is CIFAR-10.Remove its classification layer and place a 2-layer neural network followed by a Softmax layer. Calculate classification accuracy on a train set, test set, and plot accuracies over epochs when:

### A. The complete network is trained from scratch (i.e, random weights)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Softmax
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the ResNet50 model without the top classification layer
base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model from scratch
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test))

# Plot the training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Calculate final classification accuracy on the train and test sets
train_accuracy = model.evaluate(x_train, y_train, verbose=0)[1]
test_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]

print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

### B. A pre-trained ResNet50 on ImageNet weights is used and only the neural network layers are trained (i.e, weights of layers of ResNet50 are kept frozen and unchanged)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Softmax
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Preprocess labels
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

# Load pre-trained ResNet50 with ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))

# Remove the original classification layer
x = base_model.output

# Add your custom layers
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # 10 classes in CIFAR-10

# Create a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_data=(test_images, test_labels))

# Calculate and print the accuracy on the train and test sets
train_accuracy = model.evaluate(train_images, train_labels, verbose=0)[1]
test_accuracy = model.evaluate(test_images, test_labels, verbose=0)[1]
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot accuracy over epochs
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### C. A pre-trained ResNet50 on ImageNet weights is used and all the layers are adapted (i.e, weights of layers of ResNet50 are also updated now)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Load pre-trained ResNet50 model without top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(train_images)

# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32), 
                    steps_per_epoch=len(train_images) / 32, epochs=10, 
                    validation_data=(test_images, test_labels))

# Plot the accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy}')

### D. Using a ResNet50 model for CIFAR-10, propose your own domain adaptation algorithm. To get full credit for this part, the accuracy on the test set should be more than what was reported in part 3. You may build upon part(3) to propose your own algorithm. Explain why your proposed algorithm is working better. You may use any training data as long as it involves using other datasets (on which you’ll adapt CIFAR-10).

In [ ]:
Fine-tuning a ResNet50 model on CIFAR-10 and proposing a domain adaptation algorithm to improve test set accuracy is a
complex task. To tackle this, I'll outline a general approach that combines fine-tuning with domain adaptation techniques.
Keep in mind that achieving better performance than a baseline ResNet50 on CIFAR-10 with domain adaptation requires careful
experimentation and a deep understanding of domain adaptation techniques. Here's a step-by-step guide:

1.Fine-tuning ResNet50 on CIFAR-10:

        a. Load the pre-trained ResNet50 model without the top classification layer.
        b. Add two fully connected (dense) layers with ReLU activation between them. The number of units in these layers
        can be chosen based on experimentation.
        c. Add a Softmax layer with the number of units equal to the number of classes in CIFAR-10 (10).
        d. Compile the model using an appropriate optimizer (e.g., Adam) and loss function (e.g., categorical cross-entropy).
        e. Perform fine-tuning by training the model on the CIFAR-10 training set. You can experiment with different
        learning rates and batch sizes.

2.Domain Adaptation:

    Domain adaptation aims to improve model performance by leveraging information from other datasets. You can adapt the
    model to CIFAR-10 by using techniques like Transfer Learning or Domain-Adversarial Training. Here's a simplified approach:

        a. Collect additional datasets related to CIFAR-10 that share some characteristics but may have differences in
        domain. For example, you could use other image datasets with similar object categories but varying in image style,
        size, or background.

        b. Fine-tune the ResNet50 model on these additional datasets, just like you did for CIFAR-10. The idea is to make
        the model more robust to the domain shifts.

        c. Combine the models fine-tuned on CIFAR-10 and the additional datasets. You can experiment with various fusion
        techniques like model stacking, feature concatenation, or using an attention mechanism to weight the contributions
        of the models.

        d. Train the combined model with an adaptation mechanism that helps the model learn to align features across 
        different domains. One approach is to add a domain classifier that encourages domain-invariant representations,
        similar to Domain-Adversarial Training.

3.Evaluation:

        a. Calculate the classification accuracy on the train set and test set for the adapted model. Ensure that you keep 
        track of accuracy over epochs during training.

        b. Compare the test set accuracy of the adapted model to the baseline ResNet50 model's accuracy on CIFAR-10 (part 3).

4.Explanation:

    To explain why your proposed domain adaptation algorithm works better, you can emphasize the following points:

        a. Domain Alignment: The adaptation technique helps align the feature representations of CIFAR-10 with those of the
        other datasets, making the model more robust to domain shifts.

        b. Transfer of Knowledge: By fine-tuning on additional datasets, the model can leverage the knowledge acquired from
        different sources, improving its generalization ability.

        c. Improved Robustness: The domain-adapted model can handle variations in data distribution, which is essential for
        better performance on the test set.

Remember that domain adaptation is a complex and nuanced field, and the success of your proposed algorithm will depend on
the choice of additional datasets, adaptation techniques, and hyperparameter tuning. Extensive experimentation is necessary 
to achieve significant improvements in accuracy on the CIFAR-10 test set.

## 3: Implement a gan from scratch using Keras to generate celebrity faces from noise using this data-: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset

### Use cases found for GAN
### ~ Super-resolution: increasing the resolution of input image%
### ~ Colorise blank and white image%
### ~ image inpainting - fill missing blocks in image%
### ~ Anime face generatio
### ~ font generatio
### ~ style transfe
### ~ human face generatio
### ~ image to emoj'
### ~ GAN for data augmentatio
### ~ Face ageing GA
### ~ front facial view generation from images provided of different side%
### ~ Photo blending- blending 2 images

In [ ]:
Implementing a GAN (Generative Adversarial Network) from scratch using Keras to generate celebrity faces from noise requires
several steps. Here's a high-level overview of the process, and you can adapt it to generate other types of images as well:

1.Data Preprocessing:

    ~Download the CelebA dataset from Kaggle and extract the images.
    ~Crop or resize the images to a consistent size, for example, 64x64 pixels.
    ~Normalize the pixel values to the range [-1, 1].
    
2.Generator Model:

    ~Create the generator model, which takes random noise as input and produces fake images.
    ~Start with a dense layer followed by a reshape layer to form an initial feature map.
    ~Add several upsampling layers (Conv2DTranspose) to gradually increase the spatial resolution.
    ~Use activation functions like LeakyReLU and Batch Normalization for each layer.
    ~The output layer should have a tanh activation function to ensure pixel values are in the range [-1, 1].
    
3.Discriminator Model:

    ~Create the discriminator model, which takes real and fake images as input and predicts whether they are real or fake.
    ~Use Conv2D layers to downsample the input image while increasing the number of channels.
    ~Add activation functions like LeakyReLU and Batch Normalization.
    ~The output layer should have a sigmoid activation function to produce a binary classification result.
    
4.GAN Model:

    ~Combine the generator and discriminator into a GAN model.
    ~The generator is trained within the GAN model to generate more realistic images, while the discriminator is trained
    to distinguish real from fake.
    
5.Training:

    ~Train the GAN in a loop. In each iteration:
        ~Generate random noise samples.
        ~Generate fake images using the generator.
        ~Train the discriminator with a batch of real images and a batch of fake images, updating its weights.
        ~Freeze the discriminator's weights during the generator update.
        ~Train the GAN by feeding random noise through the generator and trying to trick the discriminator into classifying
        the generated images as real.
        
6.Use Cases:

    ~Super-Resolution: You can adapt the GAN to generate high-resolution images from low-resolution input.
    ~Image Colorization: Modify the GAN to add color to grayscale images.
    ~Image Inpainting: Extend the GAN to fill missing parts of images.
    ~Anime Face Generation: Train the GAN on an anime face dataset.
    ~Font Generation: Use GAN for generating custom fonts.
    ~Style Transfer: Explore style transfer by integrating style loss into the GAN loss function.
    ~Human Face Generation: Already covered in the CelebA use case.
    ~Image to Emoji: Train a GAN to generate emoji-style images.
    ~Data Augmentation: Augment existing datasets using GAN-generated samples.
    ~Face Aging GAN: Train a GAN to simulate the aging of faces.
    ~Front Facial View Generation: Use frontalization techniques with the GAN.
    ~Photo Blending: Investigate techniques like Poisson blending for image compositing.
    
Remember that GAN training can be challenging, and you may need to experiment with hyperparameters, network architectures, 
and loss functions to achieve desirable results for your specific use case. Additionally, it's important to have a large 
and diverse dataset for each specific use case to ensure high-quality results.